## 1) store data:
### 1a) import histo data into a HDF5 file

# Project name: 
    The Functional Neuroanatomy of the Human Subthalamic Nucleus, Parkinson's disease versus Controls
    
    Code by Gilles de Hollander and Max Keuken

# Goal of the project: 
    To investigate the internal organisation of the human subthalamic nucleus using a combination of histology and MRI. The data consits out of two groups: non-demented control tissue and Parkinson Disease (PD) tissue. The non-demented control tissue has been analyzed by Gilles de Hollander using the original version of this code. The goal of this code is to analyze the PD group. So the following scripts will be cleaned up to only contain the parts of code that are actually used and relevant for the project. 
### The subject ID codes that correspond to the PD data (n=10):
13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052

### The subject ID codes that correspond to the control data (n=7):
13095, 14037, 14051, 14069, 15033, 15035, 15055

# Layout of the analysis script
### 1) Combine and store the data:
    1a) import histo data into a HDF5 file that contains the histo data and the STN masks in the folder:
       /home/mkeuken1/data/post_mortem/new_data_format/
    1b) export the HDF5 file to nifti format in the folder:
       /home/mkeuken1/data/post_mortem/niftis/reoriented/
### 2) Plot the data:
    2a) load in the HDF5 data files using the base.py script. The base.py script loads in the data, sets the resolution but also smooths the data with 0.15 and 0.3mm fwhm. 
    2b) plot the CAL, PARV, MRI modality, and blockface with the available masks. At the end it also creates a correlation matrix between the stains. 
    
### 3) Statistical analysis of the mean intensity differences per stain between groups
    3a) Creating the mean intensity of the entire STN per stain, per subject
    3b) Statistical testing of the mean difference between the PD and control groups

### 4) Statistical analysis of the 27 PCA intensity sectors
    4a) Creating the 27 PCA sectors where for each stain, across the subjects we will test whether they differ from 0
    4b) Doing the actual statistical testing: t-tests which are FDR corrected for multiple comparisons.

### 5) Statistical analysis of the correlation across the stains

### 6) Statistical analysis of the correlation between stains and MRI


### 1) Combine and store the data
#### 1a) Importing the histological data as well as the masks of the STN and save them into a HDF5 file.
    No errors what so ever when running this cell

In [ ]:
# Importing a number of different tools
import re
import pandas
import glob
import h5py
import scipy as sp
from scipy import ndimage
import natsort
import numpy as np
import os

# Find the stains.png images per tissue blocks that have been registered to the blockface images
fns = glob.glob('/home/mkeuken1/data/post_mortem/stacked_slides/*/*')
reg = re.compile('.*/(?P<subject_id>[0-9]{5})_PNG/(?P<stain>[A-Za-z0-9]+)_(?P<slice>[0-9]+)_[0-9]+_(?P<id>[0-9]+)\.png')

df = pandas.DataFrame([reg.match(fn).groupdict() for fn in fns if reg.match(fn)])
df['subject_id'] = df['subject_id'].astype(int)
df['slice'] = df['slice'].astype(int)
df['fn'] = [fn for fn in fns if reg.match(fn)]
df['id'] = df['id'].astype(int)

df = df.drop_duplicates(['subject_id', 'slice', 'stain'], keep='last')

# The naming conventions of the images were a bit of a mess, so the files needed to be renamed in data frames
def correct_stain(stain):
    
    if stain == 'Cal':
        return 'CALR'
    
    if stain == 'FERR':
        return 'FER'
    
    if stain == 'Ferr':
        return 'FER'
    
    if stain == 'GABRA':
        return 'GABRA3'
    
    if stain == 'GAD':
        return 'GAD6567'
    
    if stain == 'GAD6557':
        return 'GAD6567'
    
    if stain == 'PV':
        return 'PARV'    
    
    if stain == 'SMI3':
        return 'SMI32' 
    
    if stain == 'Syn':
        return 'SYN'     

    if stain == 'TF':
        return 'TRANSF' 
    
    return stain

df['stain'] = df.stain.map(correct_stain).astype(str)

# Make a data structure that will be used for combining the MRI and histo data
df.to_pickle('/home/mkeuken1/data/post_mortem/data.pandas')

# Find the masks of the STN that were based of two raters who parcellated the STN using the PARV and SMI32 stains.
reg3 = re.compile('/home/mkeuken1/data/post_mortem/histo_masks/(?P<subject_id>[0-9]{5})_RegMasks_(?P<rater>[A-Z]+)/(?P<stain>[A-Z0-9a-z_]+)_(?P<slice>[0-9]+)_([0-9]+)_(?P<id>[0-9]+)\.png')

fns = glob.glob('/home/mkeuken1/data/post_mortem/histo_masks/*_RegMasks_*/*_*_*_*.png')

masks = pandas.DataFrame([reg3.match(fn).groupdict() for fn in fns])
masks['fn'] = fns
masks['subject_id'] = masks['subject_id'].astype(int)
masks['slice'] = masks['slice'].astype(int)

masks.set_index(['subject_id', 'slice', 'stain', 'rater'], inplace=True)
masks.sort_index(inplace=True)

masks.to_pickle('/home/mkeuken1/data/post_mortem/masks.pandas')

mask_stains = ['PARV', 'SMI32']
raters_a = ['KH', 'MT']

# There were a few masks missing (either due to not correct saving or skipping), so MCKeuken and AAlkemade parcellated the 
# remaing ones
raters_b = ['MCK', 'AA']

# A for loop that creates the .HDF5 files per tissue block 
for subject_id, d in df.groupby(['subject_id']):
    print subject_id
    
    slices = natsort.natsorted(d.slice.unique())
    
    print slices
    
    stains = natsort.natsorted(d.stain.unique())
    resolution = ndimage.imread(d.fn.iloc[0]).shape

    data_array = np.zeros((len(slices),) + resolution + (len(stains),))
    data_array[:] = np.nan
    
    print 'Storing data'
    for idx, row in d.iterrows():
        
        slice_idx = slices.index(row['slice'])
        stain_idx = stains.index(row['stain'])
        
        data_array[slice_idx, ..., stain_idx] = ndimage.imread(row.fn)
        
    mask_array = np.zeros((len(slices),) + resolution + (4,))
    
    
    print 'Storing masks'
    for idx, row in masks.ix[subject_id].reset_index().iterrows():
        
        slice_idx = slices.index(row['slice'])
        
        if row.rater in raters_a:
            last_idx = mask_stains.index(row.stain) * 2 + raters_a.index(row.rater)
        else:
            last_idx = mask_stains.index(row.stain) * 2 + raters_b.index(row.rater)
        
        im = ndimage.imread(row.fn)
        mask_array[slice_idx, ..., last_idx] = im > np.percentile(im, 70)
        
        
    print 'Creating HDF5 file'
    p = '/home/mkeuken1/data/post_mortem/new_data_format/%s/' % subject_id
    
    if not os.path.exists(p):
        os.makedirs(p)
    
    new_file = h5py.File(os.path.join(p, 'images.hdf5' % subject_id), )
    new_file.create_dataset('data', data=data_array)
    new_file.create_dataset('mask', data=mask_array.astype(bool))
    new_file.close()
    
    d.to_pickle(os.path.join(p, 'data.pandas'))
    masks.ix[subject_id].reset_index().to_pickle(os.path.join(p, 'masks.pandas'))


13044
[800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250]


### 2) Plot the data data:
#### There are two different types of plots that we are going for here. The first type is a plot that displays the intensity histogram of the stain which is combined with a tri-planner view of the STN. This is done per subject and stain. At the end of the pdf there is a correlation matrix for that given subject across the number of stains. The second type of plot is used to check whether the MRI data aligns with the blockface images, whether the stains align with the blockface images, and finally whether the masks of the STN are located in a plausible location. 

#### It should be noted that we are not using the intensity per pixel but that we smooth the data a bit. Namely with a Gaussian smoothing kernel 0.3mm fwhm. For the original analysis we also used 0.15mm fwhm. 

#### 2a) The first plot, per subject and stain: a histogram and tri-planner view followed by the correlation matrix
        this now works for both the pd and the control data

In [5]:
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

from pystain import StainDataset
import os
import numpy as np

import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

# Plot both the PD and the control data
subject_ids = [13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052, 13095, 14037, 14051, 14069, 15033, 15035, 15055]

def cmap_hist(data, bins=None, cmap=plt.cm.hot, vmin=None, vmax=None):
    n, bins, patches = plt.hist(data, bins=bins)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    
    if vmin is None:
        vmin = data.min()
    if vmax is None:
        vmax = data.max()

    # scale values to interval [0,1]
    col = (bin_centers - vmin) / vmax

    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cmap(c))


for subject_id in subject_ids[:]:
    for fwhm in [0.15, 0.3]:
        dataset = StainDataset(subject_id, fwhm=fwhm)
        dataset.get_vminmax((0, 99))

        d = '/home/mkeuken1/data/post_mortem/visualize_stains_v1/%s/' % (subject_id)
        
        if not os.path.exists(d):
            os.makedirs(d) 

        fn = os.path.join(d, 'stains_%s.pdf' % fwhm)
        pdf = PdfPages(fn)
        
        for i, stain in enumerate(dataset.stains):
            print 'Plotting %s' % stain
            plt.figure()
            # thresholded mask area is where at least 3 masks overlay
            data = dataset.smoothed_data.value[dataset.thresholded_mask, i]
            data = data[~np.isnan(data)]
            bins = np.linspace(0, dataset.vmax[i], 100)
            cmap_hist(data, bins, plt.cm.hot, vmin=dataset.vmin[i], vmax=dataset.vmax[i])
            plt.title(stain)
            plt.savefig(pdf, format='pdf')

            plt.close(plt.gcf())

            plt.figure()

            if not os.path.exists(d):
                os.makedirs(d)

            for i, orientation in enumerate(['coronal', 'axial', 'sagittal']):
                for j, q in enumerate([.25, .5, .75]):
                    ax = plt.subplot(3, 3, i + j*3 + 1)
                    slice = dataset.get_proportional_slice(q, orientation)
                    dataset.plot_slice(slice=slice, stain=stain, orientation=orientation, cmap=plt.cm.hot)
                    ax.set_anchor('NW')

                    
            plt.gcf().set_size_inches(20, 20)
            plt.suptitle(stain)
            plt.savefig(pdf, format='pdf')
            plt.close(plt.gcf())
        
        plt.figure()
        print 'Plotting correlation matrix'
        sns.heatmap(np.round(dataset.smoothed_dataframe.corr(), 2), annot=True)
        plt.savefig(pdf, format='pdf')
        plt.close(plt.gcf())

        pdf.close()

/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
data_smoothed_0.15_thr_3 not cached
 *** CALR ***
Slices that are not available for stain CALR:
 * slice 1750 (can NOT be interpolated)
 * slice 1800 (can NOT be interpolated)
 *** FER ***
Slices that are not available for stain FER:
 * slice 1800 (can be interpolated)
float64 (2240, 1768)
float64 (29, 2240, 1768, 12)
 *** GABRA3 ***
Slices that are not available for stain GABRA3:
 * slice 1300 (can NOT be interpolated)
 * slice 1400 (can NOT be interpolated)
 * slice 1800 (can NOT be interpolated)
 * slice 1850 (can NOT be interpolated)
 * slice 2150 (can be interpolated)
float64 (2240, 1768)
float64 (29, 2240, 1768, 12)
 *** GAD6567 ***
Slices that are not available for stain GAD6567:
 * slice 1100 (can be interpolated)
float64 (2240, 1768)
float64 (29, 2240, 1768, 12)
 *** MBP ***
Slices that are not available for stain MBP:
 * slice 800 (can NOT be interpolated)
 * slice 1700 (can be interpolated)
float64 (2240, 1768

/home/mkeuken1/git_projects/pystain/pystain/utils.py:19: RuntimeWarning: invalid value encountered in divide
  new_data=VV/WW


Smoothing FER
Smoothing GABRA3
Smoothing GAD6567
Smoothing MBP
Smoothing PARV
Smoothing SERT
Smoothing SMI32
Smoothing SYN
Smoothing TH
Smoothing TRANSF
Smoothing VGLUT1
calculating vmin


/opt/anaconda/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1043: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


calculating vmax
calculating vmin
calculating vmax
Plotting CALR
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting FER
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting GABRA3
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting GAD6567
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting MBP
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting PARV
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting SERT
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting SMI32
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting SYN
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting TH


/opt/anaconda/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:617: RuntimeWarning: invalid value encountered in less
  if (np.diff(bins) < 0).any():


(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting TRANSF
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting VGLUT1
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
(2240, 1768) (2240, 1768)
Plotting correlation matrix
/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
data_smoothed_0.3_thr_3 not cached
 *** CALR ***
Slices that are not available for stain CALR:
 * slice 1750 (can NOT be interpolated)
 * slice 1800 (can NOT be interpolated)
 *** FER ***
Slices that are not available for stain FER:
 * slice 1800 (can be interpolated)
float64 (2240, 1768)
float64 (29, 2240, 1768, 12)
 *** GABRA3 ***
Slices that are not available for stain GABRA3:
 * slice 1300 (can NOT be interpolated)
 * slice 1400 (can NOT be interpolated)
 * slice 1800 (can NOT be interpolated)
 * slice 1850 (can NOT be interpolated)
 * slice 2150 (can be interpolated)
float64 (2240, 1768)
float64 (29, 2240, 1768, 12)
 *** GAD656

### 2b) The second type of plot where the MRI data is plotted next to the histological data. 
#### 2b1) Per block the blockface images (im), MRI slice numberes, and the T2star weighted FLASH image (aligend to the blockface image) are loaded.
        this now works for both the pd and the control data

In [7]:
# Import some additional tools and data files
import re, glob
import pandas
import pystain
from matplotlib.backends.backend_pdf import PdfPages
import skimage
from skimage import io
import nibabel as nb


results = []

for group in ['pd','control']:
    if group == 'pd':
        subject_ids=[13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
        mri_folder='PD'

    elif group == 'control':
        subject_ids=[13095, 14037, 14051, 14069, 15033, 15035, 15055]
        mri_folder='Control'

    for subject_id in subject_ids:
        print subject_id

        year = '20' + str(subject_id)[:2]
        n = str(subject_id)[2:]

        #im = skimage.io.imread('/home/public/HISTO_TO_MRI_1/Blockface_stacks/{subject_id}/{subject_id}_rt.tif'.format(**locals()))
        #mri_slices = pandas.read_csv('/home/public/HISTO_TO_MRI_1/Blockface_stacks/True_slice_distance/{n}_{year}_3removed.txt'.format(**locals()), header=None, names=['slice'])
        im = skimage.io.imread('/home/mkeuken1/data/post_mortem/blockface_stacks/{subject_id}/{subject_id}_rt.tif'.format(**locals()))
        mri_slices = pandas.read_csv('/home/mkeuken1/data/post_mortem/blockface_stacks/true_slice_distance/{n}_{year}_3removed.txt'.format(**locals()), header=None, names=['slice'])
        
        ds = pystain.StainDataset(subject_id)

        if subject_id == None:
            r = {'subject_id':subject_id, 'bf_tif_x':im.shape[1], 'bf_tif_y':im.shape[0], 'bf_tif_slices':im.shape[2],
                 'staining_x':ds.data.shape[2], 'staining_y':ds.data.shape[1], 'staining_slices':ds.data.shape[0],             
             'n_slices_txt_file':mri_slices.shape[0]}
        else:
            mri = nb.load('/home/mkeuken1/data/post_mortem/mri/{mri_folder}/FLASH/{subject_id}/{subject_id}_{group}_flash_006_ts.nii'.format(**locals())).get_data()
            r = {'subject_id':subject_id, 'bf_tif_x':im.shape[2], 'bf_tif_y':im.shape[1], 'bf_tif_slices':im.shape[0],
                 'mri_x':mri.shape[0], 'mri_y':mri.shape[1], 'mri_n_slices':mri.shape[2],
                 'staining_x':ds.data.shape[2], 'staining_y':ds.data.shape[1], 'staining_slices':ds.data.shape[0],
                 'n_slices_txt_file':mri_slices.shape[0]}

        results.append(r)


13044
/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
13046
/home/mkeuken1/data/post_mortem/new_data_format/13046/images.hdf5
13054
/home/mkeuken1/data/post_mortem/new_data_format/13054/images.hdf5
13058
/home/mkeuken1/data/post_mortem/new_data_format/13058/images.hdf5
13060
/home/mkeuken1/data/post_mortem/new_data_format/13060/images.hdf5
13061
/home/mkeuken1/data/post_mortem/new_data_format/13061/images.hdf5
13072
/home/mkeuken1/data/post_mortem/new_data_format/13072/images.hdf5
13074
/home/mkeuken1/data/post_mortem/new_data_format/13074/images.hdf5
13077
/home/mkeuken1/data/post_mortem/new_data_format/13077/images.hdf5
15052
/home/mkeuken1/data/post_mortem/new_data_format/15052/images.hdf5
13095
/home/mkeuken1/data/post_mortem/new_data_format/13095/images.hdf5
14037
/home/mkeuken1/data/post_mortem/new_data_format/14037/images.hdf5
14051
/home/mkeuken1/data/post_mortem/new_data_format/14051/images.hdf5
14069
/home/mkeuken1/data/post_mortem/new_data_format/14069/imag

### 2b) The second type of plot where the MRI data is plotted next to the histological data. 
#### 2b2) Per block there is a pdf created which can be used to check the MRI <-> histological registration
#### This is done for the T1, T2star, and QSM seperately
     this now works for both the PD and the control data

In [8]:
for modality in ['T1', 'T2star', 'QSM'][0:]:
    for group in ['pd','control']:
        if group == 'pd':
            subject_ids=[13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
            mri_folder='PD'
            
        elif group == 'control':
            subject_ids=[13095, 14037, 14051, 14069, 15033, 15035, 15055]
            mri_folder='Control'
            
        for subject_id in subject_ids:

            print modality, subject_id

            pdf = PdfPages('/home/mkeuken1/data/post_mortem/visualize_stains_v1/{subject_id}/{modality}_stain_reg_{subject_id}.pdf'.format(**locals()))

            year = '20' + str(subject_id)[:2]
            n = str(subject_id)[2:]

            mri_slices = pandas.read_csv('/home/mkeuken1/data/post_mortem/blockface_stacks/true_slice_distance/{n}_{year}_3removed.txt'.format(**locals()), header=None, names=['slice'])
            mri_slices['slice'] = mri_slices.slice.apply(lambda e: e / 50 * 50)
            mri_slices = mri_slices.set_index('slice')

            mri_slices['n'] = np.arange(mri_slices.shape[0])

            ds = pystain.StainDataset(subject_id)

            if modality == 'T1':
                mri = nb.load('/home/mkeuken1/data/post_mortem/mri/{mri_folder}/T1_map/{subject_id}/{subject_id}_T1_2_flash_006_ts.nii.gz'.format(**locals())).get_data()

            elif modality == 'T2star':
                mri = nb.load('/home/mkeuken1/data/post_mortem/mri/{mri_folder}/T2s_map/{subject_id}/{subject_id}_T2map_2_flash_006_ts.nii.gz'.format(**locals())).get_data()

            elif modality == 'QSM':
                mri = nb.load('/home/mkeuken1/data/post_mortem/mri/{mri_folder}/QSM/{subject_id}/{subject_id}_qsm_2_flash_ts.nii.gz'.format(**locals())).get_data()            

            mri = np.swapaxes(mri, 0, 1)
            im = skimage.io.imread('/home/mkeuken1/data/post_mortem/blockface_stacks/{subject_id}/{subject_id}_rt.tif'.format(**locals()))

            if mri.shape[2] != im.shape[0]:
                im = im[3:, ...]

            assert(mri.shape[2] == im.shape[0])
            # The MRI data sometimes contained an additional rotation and this needs to be 
            # corrected to ensure that the shape of the blockface images, stains, masks and
            # MRI image are all the same:
            if subject_id in [13044]:
                mri = np.rot90(mri, 3)
            if subject_id in [13046]:
                mri = np.rot90(mri, 1)
            if subject_id in [13054]:
                mri = np.rot90(mri, 3)
            if subject_id in [13058]:
                mri = np.rot90(mri, 1)
            if subject_id in [13060]:
                mri = np.rot90(mri, 1)
            if subject_id in [13061]:
                mri = np.rot90(mri, 3)
            if subject_id in [13072]:
                mri = np.rot90(mri, 1)
            if subject_id in [13074]:
                mri = np.rot90(mri, 3)
            if subject_id in [13077]:
                mri = np.rot90(mri, 3)
            if subject_id in [13095]:
                mri = np.rot90(mri, 3)
            if subject_id in [15055]:
                mri = np.rot90(mri, 1)
            try:
                assert(mri.shape[-1] == mri_slices.shape[0])

                print 'MRI Shape: %s' % list(mri.shape)
                print 'Stain shape: %s:' %  list(ds.data.shape)
                print 'Blockface shape: %s:' % list(im.shape)

                mri_in_stain_space = np.zeros((len(ds.slices), ds.data.shape[1], ds.data.shape[2]))
                blockface_image = np.zeros((len(ds.slices), ds.data.shape[1], ds.data.shape[2], 3))

                for slice in ds.slices[:]:
                    print slice

                    if slice in mri_slices.index:

                        mri_slice_idx = mri_slices.ix[slice].n
                        stain_slice_idx = ds._get_index_slice(slice)

                        plt.subplot(141)
                        plt.imshow(sp.ndimage.gaussian_filter(ds.data[stain_slice_idx, ..., ds._get_index_stain('SMI32')], 7.5), cmap=plt.cm.inferno)
                        plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])

                        plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                        plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                        plt.title('SMI32')

                        plt.subplot(142)
                        plt.imshow(sp.ndimage.gaussian_filter(ds.data[stain_slice_idx, ..., ds._get_index_stain('PARV')], 7.5), cmap=plt.cm.inferno)
                        plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])

                        plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                        plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))    
                        plt.title('PARV')    

                        plt.subplot(143)

                        if modality == 'T2star':
                            plt.imshow(mri[:, :, mri_slice_idx], cmap=plt.cm.inferno, vmin=0, vmax=65)
                        else:
                            plt.imshow(mri[:, :, mri_slice_idx], cmap=plt.cm.inferno)
                        # plt.axis('off')
                        plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])
                        plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                        plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                        plt.title(modality)    

                        plt.subplot(144)
                        plt.imshow(im[mri_slice_idx, ...])
                        # plt.axis('off')
                        plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])
                        plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                        plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                        plt.title('Blockface image')                

                        plt.gcf().set_size_inches(40, 20)
                        plt.suptitle('Slice %d' % slice)
                        plt.savefig(pdf, format='pdf')
                        plt.close(plt.gcf())

                        mri_in_stain_space[stain_slice_idx, ...] = mri[:, :, mri_slice_idx]
                        blockface_image[stain_slice_idx, ...] = im[mri_slice_idx, ...]

                if '{modality}_in_stain_space'.format(**locals()) in ds.h5file.keys():
                    del ds.h5file['{modality}_in_stain_space'.format(**locals())]

                ds.h5file['{modality}_in_stain_space'.format(**locals())] = mri_in_stain_space
                ds.h5file['blockface_image'] = blockface_image
                ds.h5file.flush()


            except Exception as e:
                print e

            pdf.close()

T1 13044
/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
MRI Shape: [2240, 1768, 49]
Stain shape: [29, 2240, 1768, 12]:
Blockface shape: [49, 2240, 1768, 3]:
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1400
1450
1500
1550
1600
1650
1700
1750


/opt/anaconda/anaconda2/lib/python2.7/site-packages/numpy/ma/core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
T1 13046
/home/mkeuken1/data/post_mortem/new_data_format/13046/images.hdf5
MRI Shape: [2072, 1488, 48]
Stain shape: [26, 2072, 1488, 11]:
Blockface shape: [48, 2072, 1488, 3]:
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
T1 13054
/home/mkeuken1/data/post_mortem/new_data_format/13054/images.hdf5
MRI Shape: [1944, 1888, 50]
Stain shape: [23, 1944, 1888, 12]:
Blockface shape: [50, 1944, 1888, 3]:
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
T1 13058
/home/mkeuken1/data/post_mortem/new_data_format/13058/images.hdf5
MRI Shape: [2056, 1848, 53]
Stain shape: [31, 2056, 1848, 12]:
Blockface shape: [53, 2056, 1848, 3]:
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
T1 13060
/home/mkeuken1/data

KeyboardInterrupt: 

### 3) mean intensity group comparison for PD versus controls
#### 3a) For each group (PD and control), we want to have the mean intensity value per stain and test this beween groups
       Create the average intensity value within the STN per stain, per subject

In [ ]:
from sklearn.decomposition import PCA
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
import pandas as pd
from pystain import StainDataset

results = []

df = pd.DataFrame(columns=['value', 'stain', 'group', 'sub'])

for group in ['pd','control']:
        if group == 'pd':
            subject_ids=[13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
            
        elif group == 'control':
            subject_ids=[13095, 14037, 14051, 14069, 15033, 15035, 15055]

        for subject_id in subject_ids[:]:
            for fwhm in [0.3]:
                
                stain_data = StainDataset(subject_id, fwhm=fwhm)
                new_rows = pd.DataFrame(columns=['value', 'stain', 'group', 'sub'])
                
                for stain_id, stain_name in zip(np.arange(len(stain_data.stains)), stain_data.stains):
                
                    this_mean = np.nanmean(stain_data.smoothed_data[:, :, :, stain_id])
                    new_row = pd.DataFrame({'value': this_mean, 'stain': stain_name, 'group': group, 'sub': subject_id}, index=[0])
                    new_rows = new_rows.append(new_row)
                    
                df = df.append(new_rows)

#### 3b) For each group (PD and control), we now compare the two groups per stain
    First I want to have the summary stats, then I proceed to the two sample t-tests.
    where I compare the PD and control data per stain. 

In [ ]:
# 1) I first want the summary statistics (mean and std intensity) per group and stain for the entire STN
summary_stats_entire_mask = df.groupby(['group', 'stain'])['value'].agg([np.mean, np.std])
print summary_stats_entire_mask
print 'T-test pvalues'

# 2) I now want to statistically test whether the PD and controls differ in their mean intensity per stain
import scipy
for marker in df.stain.unique():
    # select the data per stain for a two sample t-test
    pd_stain_value = df.loc[(df['group'] == 'pd') & (df['stain'] == marker), 'value']
    control_stain_value = df.loc[(df['group'] == 'control') & (df['stain'] == marker), 'value']
    
    print marker
    output = scipy.stats.ttest_ind(control_stain_value,pd_stain_value,nan_policy='omit')
    print output.pvalue

### 4) Statistical analysis of the 27 PCA sectors
#### 4a) For each subject the data is collected, masked so that we only have the data in the masks, a two component PCA is run of which the first component is along the dorsal axis, whereas the second component is via the lateral axis. Then in the Y direction, or anterior/posterior axis, the structure is devided into three parts. Afterwards, for the lateral and dorsal PCA components, the line is devided into 3 parts. This is doen for each Y slices, resulting in 3x3x3: 27 sectors. 

#### The data of those 27 sectors are then combined across subjects per stain. 
    This now works for both PD and controls (the tissue blocks are treated separetly anyways but now adding a group membership label)

In [ ]:
from sklearn.decomposition import PCA
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
import pandas
from pystain import StainDataset

subject_id = 13044
ds = StainDataset(subject_id)


conversion_matrix = np.array([[0, 0, ds.xy_resolution],
                      [-ds.z_resolution, 0, 0],
                      [0, -ds.xy_resolution, 0]])
results = []

for group in ['pd','control']:
        if group == 'pd':
            subject_ids=[13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
            
        elif group == 'control':
            subject_ids=[13095, 14037, 14051, 14069, 15033, 15035, 15055]


        for subject_id in subject_ids[:]:

            ds = StainDataset(subject_id, fwhm=0.3)

        # Get coordinates of mask and bring them to mm
          https://tux06psy.fmg.uva.nl:8888/notebooks/git_projects/pystain/scripts/Analysis_pipeline_PD_HISTO.py.ipynb#  x, y, z = np.where(ds.thresholded_mask)
            coords = np.column_stack((x, y, z))

            coords_mm = conversion_matrix.dot(coords.T).T
            coords_mm -= coords_mm.mean(0)

        # Fit two components and make sure first axis walks dorsal
        # and second component lateral
            pca = PCA()
            pca.fit_transform((coords_mm - coords_mm.mean(0))[:, (0, 2)])

            components = pca.components_
            print components

            if components[0, 1] < 0:
                components[0] = -components[0]

            if components[1, 0] < 0:
                components[1] = -components[1]

            print components

            coords_dataframe = pandas.DataFrame(coords_mm, columns=['x_mm', 'y_mm', 'z_mm'])
            coords_dataframe['slice'] = x

            coords_dataframe['pc1'] = components.dot(coords_mm[:, (0, 2)].T)[0, :]
            coords_dataframe['pc2'] = components.dot(coords_mm[:, (0, 2)].T)[1, :]

            coords_dataframe[['pc1_slice_center', 'pc2_slice_center']] = coords_dataframe.groupby(['slice'])[['pc1', 'pc2']].apply(lambda x: x - x.mean())

            coords_dataframe['slice_3'] = pandas.qcut(coords_dataframe.y_mm, 3, labels=['posterior', 'middle', 'anterior'])    

            coords_dataframe['pc1_3'] = coords_dataframe.groupby('slice_3').pc1.apply(lambda d: pandas.qcut(d, 3, labels=['ventral', 'middle', 'dorsal']))
            coords_dataframe['pc2_3'] = coords_dataframe.groupby(['slice_3', 'pc1_3']).pc2.apply(lambda d: pandas.qcut(d, 3, labels=['medial', 'middle', 'lateral']))

            df= pandas.concat((ds.smoothed_dataframe, coords_dataframe), 1)
            tmp = df.pivot_table(index=['pc1_3', 'pc2_3', 'slice_3'], values=ds.stains, aggfunc='mean').copy()
            tmp['subject_id'] = subject_id
            tmp['group'] = group

            results.append(tmp.copy())

        df = pandas.concat(results).reset_index().set_index(['group','subject_id', 'slice_3', 'pc1_3', 'pc2_3'])
        df = pandas.melt(df.reset_index(), id_vars=['group','subject_id', 'slice_3', 'pc1_3', 'pc2_3'], var_name='stain')
        df['value'] = df.groupby(['group','subject_id', 'stain']).transform(lambda x: (x - x.mean()) / x.std())

        def plot_ellipse_values(values, ellipse_pars=None, size=(1000, 1000), vmin=None, vmax=None, cmap=plt.cm.coolwarm, **kwargs):

            ''' values is a n-by-m array'''


            if ellipse_pars is None:
                a = 350
                b = 150
                x = 500
                y = 500

                theta = 45. / 180 * np.pi

            else:
                a, b, x, y, theta = ellipse_pars

            A = a**2 * (np.sin(theta))**2 + b**2 * (np.cos(theta))**2
            B = 2 * (b**2 - a**2) * np.sin(theta) * np.cos(theta)
            C = a**2 * np.cos(theta)**2 + b**2 * np.sin(theta)**2
            D = -2 * A * x - B* y
            E = -B * x - 2 * C * y
            F = A* x**2 + B*x*y + C*y**2 - a**2*b**2


            X,Y = np.meshgrid(np.arange(size[0]), np.arange(size[1]))

            in_ellipse = A*X**2 + B*X*Y +C*Y**2 + D*X + E*Y +F < 0


            pc1 = np.array([[np.cos(theta)], [np.sin(theta)]])
            pc2 = np.array([[np.cos(theta - np.pi/2.)], [np.sin(theta - np.pi/2.)]])

            pc1_distance = pc1.T.dot(np.array([(X - x).ravel(), (Y - y).ravel()])).reshape(X.shape)
            pc2_distance = pc2.T.dot(np.array([(X - x).ravel(), (Y - y).ravel()])).reshape(X.shape)

            pc1_quantile = np.floor((pc1_distance / a + 1 ) / 2. * values.shape[0])
            pc2_quantile = np.floor((pc2_distance / b + 1 ) / 2. * values.shape[1])

            im = np.zeros_like(X, dtype=float)

            for pc1_q in np.arange(values.shape[0]):
                for pc2_q in np.arange(values.shape[1]):
                    im[in_ellipse * (pc1_quantile == pc1_q) & (pc2_quantile == pc2_q)] = values[pc1_q, pc2_q]


            im = np.ma.masked_array(im, ~in_ellipse)
            cax = plt.imshow(im, origin='lower', cmap=cmap, vmin=vmin, vmax=vmax, **kwargs)
            plt.grid('off')
            sns.despine()

            return cax

### 4) Statistical analysis of the 27 PCA sectors
#### 4b) For each stain and sector we do a simple t-test to compare whether the intensity values are different from zero. This is corrected for multiple comparisons using a fdr correction, critical p-value of 0.05.

#### The sectors that survive the fdr correction are then plotted on the elipsoid, where red indicates above average intensity, blue indicates below average intensity. 
    This now works for PD and controls separately

In [ ]:
from statsmodels.sandbox.stats import multicomp
from matplotlib import patches
import scipy as sp
sns.set_style('white')
df.stain.unique()

pca_folder = '/home/mkeuken1/data/post_mortem/visualize_stains_v1/PCA_sectors'
if not os.path.exists(pca_folder):
    os.makedirs(pca_folder) 
for group in ['pd','control']:

    for stain, d in df.groupby(['group','stain']):
        fn = '/home/mkeuken1/data/post_mortem/visualize_stains_v1/PCA_sectors/{stain}_big_picture_coolwarm.pdf'.format(**locals())
        pdf = PdfPages(fn)

        fig, axes = plt.subplots(nrows=1, ncols=3)

        for i, (slice, d2) in enumerate(d.groupby('slice_3')):

            ax = plt.subplot(1, 3, ['anterior', 'middle', 'posterior'].index(slice) + 1)

            n = d2.groupby(['pc1_3', 'pc2_3']).value.apply(lambda v: len(v)).unstack(1).ix[['ventral', 'middle', 'dorsal'], ['medial', 'middle', 'lateral']]
            t = d2.groupby(['pc1_3', 'pc2_3']).value.apply(lambda v: sp.stats.ttest_1samp(v, 0,nan_policy='omit')[0]).unstack(1).ix[['ventral', 'middle', 'dorsal'], ['medial', 'middle', 'lateral']]
            p = d2.groupby(['pc1_3', 'pc2_3']).value.apply(lambda v: sp.stats.ttest_1samp(v, 0,nan_policy='omit')[1]).unstack(1).ix[['ventral', 'middle', 'dorsal'], ['medial', 'middle', 'lateral']]
            mean = d2.groupby(['pc1_3', 'pc2_3']).value.mean().unstack(1).ix[['ventral', 'middle', 'dorsal'], ['medial', 'middle', 'lateral']]

            # FDR
            p.values[:] = multicomp.fdrcorrection0(p.values.ravel())[1].reshape(3, 3)

            if i == 1:
                a, b, x, y, theta  = 350, 150, 300, 275, 45
            else:
                a, b, x, y, theta  = 300, 125, 300, 275, 45.

            plot_ellipse_values(t[p<0.05].values, size=(600, 550), ellipse_pars=(a, b, x, y,  theta / 180. * np.pi), vmin=-7, vmax=7, cmap=plt.cm.coolwarm)


            e1 = patches.Ellipse((x, y), a*2, b*2,
                             angle=theta, linewidth=2, fill=False, zorder=2)

            ax.add_patch(e1)

            plt.xticks([])
            plt.yticks([])    

            sns.despine(bottom=True, left=True)

            #sns.despine(bottom=True, left=True)
            print stain
            print p.values  
        plt.suptitle(stain, fontsize=24)
        fig.set_size_inches(15., 4.)
        pdf.savefig(fig, transparent=True)    
        pdf.close()

 

### from here work in progress to get the correlation matrices between stains
       compare it with the submitted paper, note that the PCA stains I could replicate identically

In [ ]:
from pystain import StainCluster, StainDataset
import h5py
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import pandas
import scipy as sp
from scipy import spatial
import scipy.cluster.hierarchy as hc
from statsmodels.sandbox.stats import multicomp

corr_folder = '/home/mkeuken1/data/post_mortem/visualize_stains_v1/correlations'
if not os.path.exists(corr_folder):
    os.makedirs(corr_folder) 
    
for group in ['control']:#'pd','control']:
        if group == 'pd':
            subject_ids=[13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
            
        elif group == 'control':
            subject_ids=[13095, 14037, 14051, 14069, 15033, 15035, 15055]

        corrs = []
        pdf = PdfPages('/home/mkeuken1/data/post_mortem/visualize_stains_v1/correlations/{group}_hierachical_clusters.pdf'.format(**locals()))

        for subject_id in subject_ids[:]:
            dataset = StainDataset(subject_id, fwhm=0.3)
            corr = dataset.smoothed_dataframe.corr()
            # Fill the correlation matrix data frame
            corrs.append(corr)
        
        
        corr_matrix = np.array([corr.values for corr in corrs])
               
        # Stat testing
        t, p = sp.stats.ttest_1samp(corr_matrix, 0)
        np.fill_diagonal(p, np.nan)
        plt.hist(p[~np.isnan(p)].ravel(), bins=np.arange(0, 1, 0.01))
        
        p_mask = ~np.isnan(p)
        _, p_corrected_ravel = multicomp.fdrcorrection0(p[p_mask].ravel(), alpha=0.05)
        p_corrected = np.zeros_like(p)
        p_corrected[p_mask] = p_corrected_ravel

        mean_rs = np.mean([corr.values for corr in corrs],0)
        #mean_rs = np.nan_to_num(mean_rs)

        mean_rs = pandas.DataFrame(mean_rs, columns=dataset.stains, index=dataset.stains)
        #sns.set_context('poster')
        #sns.heatmap(mean_rs, annot=True, )

        DF_dism = 1 - mean_rs   # distance matrix

        linkage = hc.linkage(sp.spatial.distance.squareform(DF_dism, checks=False), method='average' )
        mean_rs_ = mean_rs.copy()
        np.fill_diagonal(mean_rs_.values, 0)
        cg = sns.clustermap(np.round(mean_rs, 2), row_linkage=linkage, col_linkage=linkage, cmap='coolwarm', annot=True)
        plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)

        stain_order = [t.get_text() for t in cg.ax_heatmap.get_xticklabels()]

        plt.gcf().set_size_inches(15, 15)
        plt.savefig(pdf, format='pdf')
            
        pdf.close()     

In [ ]:
subject_ids = [13095, 14037, 14051, 14069, 15033, 15035, 15055]

In [ ]:
from pystain import StainCluster, StainDataset

In [ ]:
import h5py

In [ ]:
f = h5py.File('/home/mkeuken1/data/post_mortem/new_data_format/13095/images.hdf5')

In [ ]:
corrs = []

for subject_id in subject_ids[:]:
    dataset = StainDataset(subject_id, fwhm=0.3)
    corr = dataset.smoothed_dataframe.corr()
    corrs.append(corr)

In [ ]:
corr_matrix = np.array([corr.values for corr in corrs])

In [ ]:
t, p = sp.stats.ttest_1samp(corr_matrix, 0)
np.fill_diagonal(p, np.nan)
plt.hist(p[~np.isnan(p)].ravel(), bins=np.arange(0, 1, 0.01))
# note the difference in histogram...

In [ ]:
import statsmodels

In [ ]:
from statsmodels.sandbox.stats import multicomp

In [ ]:
p_mask = ~np.isnan(p)

In [ ]:
_, p_corrected_ravel = multicomp.fdrcorrection0(p[p_mask].ravel(), alpha=0.05)
p_corrected = np.zeros_like(p)
p_corrected[p_mask] = p_corrected_ravel

In [ ]:
import pandas

In [ ]:
mean_rs = np.mean([corr.values for corr in corrs],0)
# mean_rs = np.ma.masked_array(mean_rs, p_corrected > 0.05)
mean_rs = pandas.DataFrame(mean_rs, columns=dataset.stains, index=dataset.stains)

In [ ]:
sns.set_context('poster')

In [ ]:
sns.heatmap(mean_rs, annot=True, )
# note that these are slightly different correlations than the ones reported in the elife manuscript
# also note that the correlation matrices as plotted in the data checking pdf are also slightly different. 
# whereas the raw histogram data seems to be identical..

# The differences in code?
# the base.py that I'm using compared to the one in gilles his home directory has two extra lines:
#		print new_slice.dtype, new_slice.shape
#		print self.data.dtype, self.data.shape

# This shouldnt do anything as it basically only prints stuff to the terminal
# the cluster.py that I'm using is identical to the one in gilles his home directory

# Talked to Gilles it might be the case that I included more slices than he did as Anneke might have given him a list of 
# poorly registered slices